This notebook is meant to demonstrate the use of quantile parameterization. Basic usage is covered in the first section. In the second half two different classes of distribution are used to showcase how the various PDF reconstruction algorithms perform.

In [ ]:
# Imports and global defaults
import qp
import numpy as np
import matplotlib.pyplot as plt

# Sets the x axis limits on all plots, rerun notebook to apply changes.
STD_X_LIM = (0,5)

In [ ]:
# Create an Ensemble with a single normal distribution
single_norm = qp.Ensemble(qp.stats.norm, data=dict(loc=3, scale=0.5))

# Starting with a simple base case. Reasonable quantiles >0, <1.0
single_norm_quantiles = np.linspace(0.001, 0.999, 16)
single_norm_locations = single_norm.ppf(single_norm_quantiles)

# Create a quantile parameterization ensemble
quant_dist_single_norm = qp.quant(quants=single_norm_quantiles, locs=single_norm_locations)

In [ ]:
# Print out the constructor name
print(quant_dist_single_norm.dist.pdf_constructor_name)

In [ ]:
# Create an Ensemble with 3 normal distributions
means = np.array([[1], [2.5], [3]])
scales = np.array([[0.25], [0.5], [0.1]])
many_norm = qp.Ensemble(qp.stats.norm, data=dict(loc=means, scale=scales))

# Starting with a simple base case. Reasonable quantiles >0, <1.0
many_norm_quantiles = np.linspace(0.001, 0.999, 16)
many_norm_locations = many_norm.ppf(many_norm_quantiles)

# Create a quantile parameterization ensemble
quant_dist_many_norm = qp.quant(quants=many_norm_quantiles, locs=many_norm_locations)

# The following approach is identical, and is basically syntactical sugar.
quant_dist_many_norm = qp.convert(many_norm, 'quant', quants=many_norm_quantiles)

In [ ]:
# Create a grid and call `pdf` to return the y values to reconstruct the initial PDF
user_provided_grid = np.linspace(0,4,100)
results = quant_dist_many_norm.pdf(user_provided_grid)
print(results.shape)

In [ ]:
user_provided_grid = np.linspace(0,4,100)

# Use the following syntax to retrieve a specific distribution's PDF from the Ensemble
results = quant_dist_many_norm[1].pdf(user_provided_grid)
print(results.shape)

In [ ]:
# Show that we can use different row values, and get different values out.
user_provided_grid = np.linspace(0,4,100)
user_provided_row = [0]
result_1 = quant_dist_many_norm.dist.pdf_constructor.construct_pdf(grid=user_provided_grid, row=user_provided_row)

user_provided_row = [1]
result_2 = quant_dist_many_norm.dist.pdf_constructor.construct_pdf(grid=user_provided_grid, row=user_provided_row)

# Expect that this will be non-zero
print(np.sum(result_1 - result_2))

In [ ]:
# Show that you can pass in multiple row index values
user_provided_grid = np.linspace(0,4,100)
user_provided_rows = [[0,1]]
results = quant_dist_many_norm.dist.pdf_constructor.construct_pdf(grid=user_provided_grid, row=user_provided_rows)

# Expect this to be (2,100), but the `.ravel()` function in the piecewise_*
# constructors convert the results of interpolation into an array with shape = (2*100,)...
print(results.shape)

In [ ]:
# Show basic plotting
_, ax1 = qp.plotting.plot_native(quant_dist_single_norm, xlim=STD_X_LIM)
qp.plotting.plot_native(single_norm, axes=ax1)
ax1.set_title('Single distribution quantile parameterization')

plot_index = 0
_, ax2 = qp.plotting.plot_native(quant_dist_many_norm[plot_index], xlim=STD_X_LIM)
qp.plotting.plot_native(many_norm[plot_index], axes=ax2)
ax2.set_title('Single selected distribution from Ensemble of quantile parameterization')

## Examination of different reconstruction algorithms for various types of distributions

### Simple Gaussian

In [ ]:
# Same as the single_norm Ensemble created at the beginning of the notebook.
single_norm = qp.Ensemble(qp.stats.norm, data=dict(loc=3, scale=0.5))
fig, ax = qp.plotting.plot_native(single_norm, xlim=STD_X_LIM)
ax.set_title('Original simple Gaussian distribution')

In [ ]:
# Define the quantiles, extract the locations from the original distribution
single_norm_quantiles = np.linspace(0.001, 0.999, 16)
single_norm_locations = single_norm.ppf(single_norm_quantiles)

# Create a quantile parameterization ensemble
quant_dist_single_norm = qp.quant(quants=single_norm_quantiles, locs=single_norm_locations)

In [ ]:
fig, ax = plt.subplots(2,2)
plt.setp(ax, xlim=STD_X_LIM)

quant_dist_single_norm.dist.pdf_constructor_name = 'piecewise_linear' # Will emit a logger warning that this constructor is already selected
qp.plotting.plot_native(quant_dist_single_norm, axes=ax[0,0])
qp.plotting.plot_native(single_norm, axes=ax[0,0])


quant_dist_single_norm.dist.pdf_constructor_name = 'piecewise_constant'
qp.plotting.plot_native(quant_dist_single_norm, axes=ax[0,1])
qp.plotting.plot_native(single_norm, axes=ax[0,1])


quant_dist_single_norm.dist.pdf_constructor_name = 'cdf_spline_derivative'
qp.plotting.plot_native(quant_dist_single_norm, axes=ax[1,0])
qp.plotting.plot_native(single_norm, axes=ax[1,0])


quant_dist_single_norm.dist.pdf_constructor_name = 'dual_spline_average'
qp.plotting.plot_native(quant_dist_single_norm, axes=ax[1,1])
qp.plotting.plot_native(single_norm, axes=ax[1,1])

ax[0,0].set_title('piecewise_linear')
ax[0,1].set_title('piecewise_constant')
ax[1,0].set_title('cdf_spline_derivative')
ax[1,1].set_title('dual_spline_average')

### Simple Rayleigh

In [ ]:
# Same as the single_norm Ensemble created at the beginning of the notebook.
single_rayleigh = qp.Ensemble(qp.stats.rayleigh, data=dict(loc=0, scale=1))
fig, ax = qp.plotting.plot_native(single_rayleigh, xlim=STD_X_LIM)
ax.set_title('Original simple Rayleigh distribution')

In [ ]:
# Define the quantiles, extract the locations from the original distribution
single_rayleigh_quantiles = np.linspace(0.001, 0.999, 16)
single_rayleigh_locations = single_rayleigh.ppf(single_rayleigh_quantiles)

# Create a quantile parameterization ensemble
quant_dist_single_rayleigh = qp.quant(quants=single_rayleigh_quantiles, locs=single_rayleigh_locations)

In [ ]:
fig, ax = plt.subplots(2,2)
plt.setp(ax, xlim=STD_X_LIM)

quant_dist_single_rayleigh.dist.pdf_constructor_name = 'piecewise_linear' # Will emit a logger warning that this constructor is already selected
qp.plotting.plot_native(quant_dist_single_rayleigh, axes=ax[0,0])
qp.plotting.plot_native(single_rayleigh, axes=ax[0,0])


quant_dist_single_rayleigh.dist.pdf_constructor_name = 'piecewise_constant'
qp.plotting.plot_native(quant_dist_single_rayleigh, axes=ax[0,1])
qp.plotting.plot_native(single_rayleigh, axes=ax[0,1])


quant_dist_single_rayleigh.dist.pdf_constructor_name = 'cdf_spline_derivative'
qp.plotting.plot_native(quant_dist_single_rayleigh, axes=ax[1,0])
qp.plotting.plot_native(single_rayleigh, axes=ax[1,0])


quant_dist_single_rayleigh.dist.pdf_constructor_name = 'dual_spline_average'
qp.plotting.plot_native(quant_dist_single_rayleigh, axes=ax[1,1])
qp.plotting.plot_native(single_rayleigh, axes=ax[1,1])

ax[0,0].set_title('piecewise_linear')
ax[0,1].set_title('piecewise_constant')
ax[1,0].set_title('cdf_spline_derivative')
ax[1,1].set_title('dual_spline_average')

### Merged Gaussian distributions

In [ ]:
# Create a multi-modal distribution where two Gaussian distributions are merged
mu =  np.array([1.7, 2.9])
sig = np.array([0.3, 0.4])
wt =  np.array([0.4, 0.6])

merged_norms = qp.Ensemble(qp.mixmod, data=dict(means=mu, stds=sig, weights=wt))
fig, ax = qp.plotting.plot_native(merged_norms, xlim=STD_X_LIM)
ax.set_title('Original merged Gaussian distribution')

In [ ]:
# Define the quantiles, extract the locations from the original distribution
merged_norm_quantiles = np.linspace(0.001, 0.999, 30)
merged_norm_locations = merged_norms.ppf(merged_norm_quantiles)

# Create a quantile parameterization Ensemble
quant_dist_merged_norm = qp.quant(quants=merged_norm_quantiles, locs=merged_norm_locations)

The following plots show how each of the different pdf reconstruction algorithms perform in the case of a merged distribution.

In [ ]:
fig, ax = plt.subplots(2,2)
plt.setp(ax, xlim=STD_X_LIM)

quant_dist_merged_norm.dist.pdf_constructor_name = 'piecewise_linear' # Will emit a logger warning that this constructor is already selected
qp.plotting.plot_native(quant_dist_merged_norm, axes=ax[0,0])
qp.plotting.plot_native(merged_norms, axes=ax[0,0])


quant_dist_merged_norm.dist.pdf_constructor_name = 'piecewise_constant'
qp.plotting.plot_native(quant_dist_merged_norm, axes=ax[0,1])
qp.plotting.plot_native(merged_norms, axes=ax[0,1])


quant_dist_merged_norm.dist.pdf_constructor_name = 'cdf_spline_derivative'
qp.plotting.plot_native(quant_dist_merged_norm, axes=ax[1,0])
qp.plotting.plot_native(merged_norms, axes=ax[1,0])


quant_dist_merged_norm.dist.pdf_constructor_name = 'dual_spline_average'
qp.plotting.plot_native(quant_dist_merged_norm, axes=ax[1,1])
qp.plotting.plot_native(merged_norms, axes=ax[1,1])

ax[0,0].set_title('piecewise_linear')
ax[0,1].set_title('piecewise_constant')
ax[1,0].set_title('cdf_spline_derivative')
ax[1,1].set_title('dual_spline_average')

### Distinct Gaussian distributions

In [ ]:
# Create a multi-modal distribution with two distinct Gaussian distributions
mu =  np.array([1, 3])
sig = np.array([0.1, 0.1])
wt =  np.array([0.4, 0.6])

distinct_norms = qp.Ensemble(qp.mixmod, data=dict(means=mu, stds=sig, weights=wt))
fig, ax = qp.plotting.plot_native(distinct_norms, xlim=STD_X_LIM)
ax.set_title('Original distinct Gaussian distribution')

In [ ]:
# Define the quantiles, extract the locations from the original distribution
distinct_norm_quantiles = np.linspace(0.001, 0.999, 30)
distinct_norm_locations = distinct_norms.ppf(distinct_norm_quantiles)

# Create a quantile parameterization Ensemble
quant_dist_distinct_norm = qp.quant(quants=distinct_norm_quantiles, locs=distinct_norm_locations)

In [ ]:
fig, ax = plt.subplots(2,2)
plt.setp(ax, xlim=STD_X_LIM)

quant_dist_distinct_norm.dist.pdf_constructor_name = 'piecewise_linear' # Will emit a logger warning that this constructor is already selected
qp.plotting.plot_native(quant_dist_distinct_norm, axes=ax[0,0])
qp.plotting.plot_native(distinct_norms, axes=ax[0,0])

quant_dist_distinct_norm.dist.pdf_constructor_name = 'piecewise_constant'
qp.plotting.plot_native(quant_dist_distinct_norm, axes=ax[0,1])
qp.plotting.plot_native(distinct_norms, axes=ax[0,1])

quant_dist_distinct_norm.dist.pdf_constructor_name = 'cdf_spline_derivative'
qp.plotting.plot_native(quant_dist_distinct_norm, axes=ax[1,0])
qp.plotting.plot_native(distinct_norms, axes=ax[1,0])

quant_dist_distinct_norm.dist.pdf_constructor_name = 'dual_spline_average'
qp.plotting.plot_native(quant_dist_distinct_norm, axes=ax[1,1])
qp.plotting.plot_native(distinct_norms, axes=ax[1,1])

ax[0,0].set_title('piecewise_linear')
ax[0,1].set_title('piecewise_constant')
ax[1,0].set_title('cdf_spline_derivative')
ax[1,1].set_title('dual_spline_average')

## PDF Reconstruction algorithm `debug` method
Each PDF reconstruction class has a `debug` method which will generally return a tuple of the primary and intermediate calculations used to reconstruct the original PDF. 

In [ ]:
# Create a multi-modal distribution where two Gaussian distributions are merged - same as the section above, "Merged Gaussian distributions".
mu =  np.array([1.7, 2.9])
sig = np.array([0.3, 0.4])
wt =  np.array([0.4, 0.6])

merged_norms = qp.Ensemble(qp.mixmod, data=dict(means=mu, stds=sig, weights=wt))
fig, ax = qp.plotting.plot_native(merged_norms, xlim=STD_X_LIM)
ax.set_title('Original merged Gaussian distribution')



In the following diagram, we've used the `debug` method to retrieve the intermediate calculations from the dual spline average PDF reconstructor. 

The plot shows the original PDF in blue, and the reconstructed points as orange stars. Shown as a green dotted line, the trapezoid reconstruction of the original PDF uses an iterative approach to calculate the unknown corner of the trapezoid approximation of the curve.

Noting that the trapezoid reconstruction oscillates around the true PDF at every point leads to fitting two splines (the purple and red dashed lines) on the even and odd points of the trapezoid reconstruction. The final output is the average of the two splines at a given x location.

In [ ]:
# Define the quantiles, extract the locations from the original distribution
merged_norm_quantiles = np.linspace(0.001, 0.999, 30)
merged_norm_locations = merged_norms.ppf(merged_norm_quantiles)

# Create a quantile parameterization Ensemble
quant_dist_merged_norm = qp.quant(quants=merged_norm_quantiles, locs=merged_norm_locations)

# Set the PDF reconstruction algorithm to use
quant_dist_merged_norm.dist.pdf_constructor_name = 'dual_spline_average'

# Retrieve the reconstructed PDF manually by passing in a "grid".
x_values = np.linspace(0,5,100)
original_y_values = merged_norms.pdf(x_values)
y_values = quant_dist_merged_norm.pdf(x_values)

# Retrieve the primary data and intermediate calculations for the algorithm.
# Here `quantiles`` and `locations`` are the primary data. `p_of_zs`, `y1`, and `y2` are intermediate results.
# `p_of_zs` represents the intermediate stepwise calculation of area under the PDF based on the the difference between adjacent quantiles
# `y1` and `y2` are splines fit to the odd and even `p_of_zs` values (respectively)
quantiles, locations, p_of_zs, y1, y2 = quant_dist_merged_norm.dist.pdf_constructor.debug()

plt.plot(x_values, original_y_values, linewidth=5, label='Original PDF')
plt.plot(x_values, y_values, marker='*', linestyle='none', label='Reconstructed PDF')
plt.plot(np.squeeze(locations), np.squeeze(p_of_zs), linestyle=':', label='Intermediate trapezoid')
plt.plot(x_values, np.squeeze(y1), linestyle='--', label='Lower spline fit')
plt.plot(x_values, np.squeeze(y2), linestyle='--', label='Upper spline fit')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlim(0,5)
